In [1]:
import torch
import src

In [2]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

__CUDNN VERSION: 7605
__Number CUDA Devices: 1
__CUDA Device Name: GeForce GTX 1060
__CUDA Device Total Memory [GB]: 6.373572608
Device:  cuda


In [3]:
from sklearn.preprocessing import LabelEncoder
from datasets import ucr_2018, utils

train_df, test_df = ucr_2018.get_dataset("ECG200")
X_train, y_train = utils.x_y_split(train_df, y_col=0)
X_test, y_test = utils.x_y_split(test_df, y_col=0)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [4]:
params = {
    'batch_size': 10,
    'shuffle': True
}

train = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.values).unsqueeze(1).float().to(device),
    torch.tensor(y_train).to(device)
)
train_loader = torch.utils.data.DataLoader(train, **params)

test = torch.utils.data.TensorDataset(
    torch.from_numpy(X_test.values).unsqueeze(1).float().to(device),
    torch.tensor(y_test).to(device)
)
test_loader = torch.utils.data.DataLoader(test, **params)

In [5]:
from src.cnn_classifier import CNNClassifier

epochs = 100
model = CNNClassifier(
        lr=1e-4,
        Optimizer=torch.optim.Adam,
        device=device
).to(device)
model.train(train_loader=train_loader, validation_loader=test_loader,
            epochs=epochs, model_name="CNN_classifier", logdir="logs",
            verbose=1)

Epoch 0, dataset train: Loss - 0.708077, F1-score - 0.000000
Epoch 0, dataset validation: Loss - 0.700289, F1-score - 0.000000
Epoch 1, dataset train: Loss - 0.701117, F1-score - 0.000000
Epoch 1, dataset validation: Loss - 0.695209, F1-score - 0.000000
Epoch 2, dataset train: Loss - 0.695310, F1-score - 0.205128
Epoch 2, dataset validation: Loss - 0.690081, F1-score - 0.720721
Epoch 3, dataset train: Loss - 0.688037, F1-score - 0.849315
Epoch 3, dataset validation: Loss - 0.685701, F1-score - 0.810127
Epoch 4, dataset train: Loss - 0.682405, F1-score - 0.816568
Epoch 4, dataset validation: Loss - 0.680927, F1-score - 0.780488
Epoch 5, dataset train: Loss - 0.677326, F1-score - 0.816568
Epoch 5, dataset validation: Loss - 0.675911, F1-score - 0.780488
Epoch 6, dataset train: Loss - 0.670357, F1-score - 0.816568
Epoch 6, dataset validation: Loss - 0.671828, F1-score - 0.780488
Epoch 7, dataset train: Loss - 0.664470, F1-score - 0.816568
Epoch 7, dataset validation: Loss - 0.668125, F1-s

In [7]:
#!rm -rf ./logs/CNN_clasifier